In [1]:
%matplotlib inline

import numpy as np
from numpy import random, nanmax, argmax, unravel_index
import cv2
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

class Plant():
    def __init__(img, **kwargs):
        self.blur = None
        self.gradient = None
        self.binary = None
        self.foreground = None
        self.h = None
        self.w = None
        self.background = None
        self.markers = None
        self.marker_opts = None
        
        # Inputs: image, gaussian blur kernel size, gaussian blur sigma, morphological gradient kernel size, 
    # lower bound and upper bound for binary gradient    
    def setup_watershed(img, gb_sz, gb_sig, mg_sz, lval, uval):
        blur = cv2.GaussianBlur(img, (gb_sz, gb_sz), gb_sig)
    #     plot(blur, "Blurred", 10)
        h, w = img.shape[:2]

        # morphological gradient
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (mg_sz, mg_sz))
        gradient = cv2.morphologyEx(blur, cv2.MORPH_GRADIENT, kernel)
    #     plot(gradient, "Morphological gradient", 10)

        # Binarize gradient
        lowerb = np.array([lval, lval, lval])
        upperb = np.array([uval, uval, uval])
        binary = cv2.inRange(gradient, lowerb, upperb)
        plot_gray(binary, "Binarized gradient", 10, "gray")

        # Flood fill
        for row in range(h):
            if binary[row, 0] == 255:
                cv2.floodFill(binary, None, (0, row), 0)
            if binary[row, w-1] == 255:
                cv2.floodFill(binary, None, (w-1, row), 0)

        for col in range(w):
            if binary[0, col] == 255:
                cv2.floodFill(binary, None, (col, 0), 0)
            if binary[h-1, col] == 255:
                cv2.floodFill(binary, None, (col, h-1), 0)
                
#         plot_gray(binary, "Flood fill from edges", 10, "gray")

        # Cleaning up mask
        foreground = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)
        foreground = cv2.morphologyEx(foreground, cv2.MORPH_CLOSE, kernel)

        # Cleans up foreground
        fig = plt.figure(figsize=(10,10))
        plt.imshow(foreground, cmap = 'gray')
        plt.title("Foreground mask")
        plt.xticks([])
        plt.yticks([])
        return blur, gradient, binary, foreground, h, w;

    def draw_watershed(img, foreground, h, w, k_sz, background_it, hue_max):
        # Creating background and unknown mask for labeling
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (k_sz, k_sz))
        background = cv2.dilate(foreground, kernel, iterations = background_it)
        unknown = cv2.subtract(background, foreground)
#         plot_gray(background, "Background mask", 10, "gray")

        # Watershed
        markers = cv2.connectedComponents(foreground)[1]
        markers += 1  # Add one to all labels so that background is 1, not 0
        markers[unknown==255] = 0  # mark the region of unknown with zero
        markers = cv2.watershed(img, markers)

        # Assign the markers a hue between 0 and hue_max
        hue_markers = np.uint8(hue_max*np.float32(markers)/np.max(markers))
        blank_channel = 255*np.ones((h, w), dtype=np.uint8)
        marker_img = cv2.merge([hue_markers, blank_channel, blank_channel])
        marker_img = cv2.cvtColor(marker_img, cv2.COLOR_HSV2BGR)
    #     plot(marker_img, "Markered image", 10)

        # Label the original image with the watershed markers
        labeled_img = img.copy()
        labeled_img[markers>1] = marker_img[markers>1]  # 1 is background color
        labeled_img = cv2.addWeighted(img, 0.5, labeled_img, 0.5, 0)
    #     plot(labeled_img, "Labeled with watershed markers", 10)
        return background, markers;
    
    def det_opts(markers):
        unq, ct = np.unique(markers, return_counts = True)
        # Determine the avg number time a marker is shown
        avg = sum(ct) / len(ct) 
        print(avg)
        val = ct > avg

        # Explore these values 
        opt = unq[val]
        return opt;

    # Inputs: markers, marker value to look for, kernel size, number of iterations
    def marker_test(markers, val, k_sz, n):
        markers1 = ((markers == val)).astype(np.uint8) 
        ret, m = cv2.threshold(markers1, 0, 255, cv2.THRESH_BINARY|cv2.THRESH_OTSU)
        kernel = np.ones((k_sz, k_sz), np.uint8) 
        dm = cv2.dilate(m, kernel, iterations = n)
        em = cv2.erode(dm, kernel, iterations = n)

        titles = ['Original', 'Dilated', 'Eroded']
        all = [m, dm, em]

        fig = plt.figure(figsize=(10,10))
        for i in range(3):
            fig.add_subplot(1,3,i+1), plt.imshow(all[i])
            plt.title(titles[i])
            plt.xticks([]),plt.yticks([])
        plt.show()

        return em;    

    def get_contours(img, em):
        contours, hierarchy = cv2.findContours(em, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
        # access the array and then -1 implies to draw all the contours indicated by c
        for c in contours:
            cv2.drawContours(img, c, -1, (0, 255, 0), 7)
        #plot(img, "Image with Contours Drawn", 10)
        return contours;